In [ ]:
# importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# reading tour dataframes
df = pd.read_csv(r'..\Tour data\all_Tours.csv')

In [ ]:
# showing the first 5 rows
df.head()

In [ ]:
# information about the dataframe
df.info()

**Issues**

- Remove Unnamed: 0 column
- Drop the null values
- Remove $ from Tour Price and convert it to float
- Replace Image column before the link
- Rename the Link column (Agency Link)
- Tour Duration convert different unit into one unit (in hour) deal with the different format of the time value.

In [ ]:
# making copy for cleaning
df_clean = df.copy()

## Cleaning 

*1. General cleaning* <br>
*1.1 Drop the null values*

In [ ]:
# drop the null value
df_clean = df_clean.dropna()

*1.2 Rename Tour Name, Tour Type, Tour Duration, Tour Price & Link columns*

In [ ]:
# renaming the columns
df_clean.rename(columns={'Tour Name':'Tour', 'Tour Type':'Type', 'Tour Duration':'Duration', 'Tour Price':'Price', 'Link':'Link to agency'}, inplace=True)

In [ ]:
# check the above
df_clean.info()

*2. Specific column cleaning*
<br>
*2.1 Remove unnecessary column*

In [ ]:
# drop the Unnamed: 0 column
df_clean.drop('Unnamed: 0', axis=1, inplace=True)

*2.2 Image column*

In [ ]:
# Replace the image column
df_clean = df_clean[['Tour', 'Type', 'Duration', 'Price', 'Price Per', 'Cancellation','Tour By', 'City', 'Country', 'Image', 'Link to agency']]

*2.3 Tour Name column*

In [ ]:
# check the number of unique values in Tour name
df_clean['Tour'].nunique()

*Since the number of unique value in the Tour column is not the same number of entries in the dataset. This mean there are so many duplicates in the name. Whereas, the Tour column should be all unique.*

In [ ]:
# check the value counts in Tour name
df_clean['Tour'].value_counts()

In [ ]:
# drop the duplicates in Tour column
df_clean.drop_duplicates(subset=['Tour'],inplace=True)

In [ ]:
# to check the above
df_clean.info()

*2.4 Tour Type column*

In [ ]:
# check the number of unique values in Tour Type
df_clean['Type'].nunique()

In [ ]:
# Bar chart for the top 10 Type
x = df_clean['Type'].value_counts().head(10).index
y = df_clean['Type'].value_counts().head(10)

plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20,10))
plt.xticks( color ='#000000',fontsize=12, rotation = 15)
plt.yticks(color ='#000000',fontsize=12)
plt.bar(x, y, color= ('#458B74'), width=0.6)
plt.xlabel('Tour Type',fontsize=20, color ='#000000')
plt.ylabel('Number of tour',fontsize=20, color ='#000000')
plt.title('The top 10 Tour Types',fontsize=22, color ='#000000')
plt.show();

*Bus Tours is the top type of tour in thid dataset, with more than 3500 Tours.  Followwed by the walking Tours*

*2.5 Tour Duration column*

In [ ]:
# check the number of unique values in Tour Duration
df_clean['Duration'].nunique()

In [ ]:
# check the value count in Tour Duration
df_clean['Duration'].value_counts()

*The values in this column are in different units: days, hours and minutes.*
*So, let separate the units first and then convert them all inot one unit (in hour).*
*The values with range I will calculate the average of them. As what I saw back in the tripadvisor.com.*

*2.5.1 Separate the time value and the duration unit in two columns*

In [ ]:
# separate each the duration (in number) and the unit making new columns
df_clean[['Time','Unit']] = df_clean.Duration.str.split(" ",expand=True)
df_clean

*2.5.2 Removing other characters in Time column*

In [ ]:
# removing the + sign from Time column, by using str.replace
df_clean['Time'] = df_clean['Time'].str.replace('+', '')
df_clean['Time'] = df_clean['Time'].str.replace(',', '')

*After removing + sign and the comma , now I can separate the other format in Time column* <br> 
<br>
*2.5.3 Separate the numbers in Time column to minimum duration & maximum duration*

In [ ]:
# separate each the Time value in XX-XX this format to two column minD (minimum Duration) & maxD (maximum Duration)
df_clean[['minD','maxD']] = df_clean.Time.str.split("–",expand=True)
df_clean

*2.5.4 Convert data type to float*

In [ ]:
# convert the data type of minD & maxD to float
df_clean['minD'] = pd.to_numeric(df_clean['minD'], downcast="float")
df_clean['maxD'] = pd.to_numeric(df_clean['maxD'], downcast="float")

*2.5.5 Calculate time average*

In [ ]:
# fill the null value in maxD with the same vale in minD for later on average
df_clean['maxD'].fillna(df_clean['minD'], inplace=True)

In [ ]:
# calculating the average Time
df_clean['AverageTime'] = (df_clean['minD'] + df_clean['maxD']) / 2

In [ ]:
# drop the Duration, Time, minD & maxD columns
df_clean.drop(['Duration', 'Time', 'minD','maxD'], axis=1, inplace=True)

*2.5.6 Convert time average to hours*

In [ ]:
# Convert days to hours
df_clean.loc[df_clean['Unit']=='days', ['AverageTime']] *= 24
df_clean.loc[df_clean['Unit']=='days', ['AverageTime']] *= 24

# Convert minutes to hours
df_clean.loc[df_clean['Unit']=='minutes', ['AverageTime']] /= 60
df_clean.loc[df_clean['Unit']=='minute', ['AverageTime']] /= 60

# make sure hour will be the same
df_clean.loc[df_clean['Unit']=='hours', ['AverageTime']] *= 1
df_clean.loc[df_clean['Unit']=='hour', ['AverageTime']] *= 1

*2.5.7 Rename, drop and replace columns after convertaing to average time in hour*

In [ ]:
# Rename AverageTime column
df_clean.rename(columns={'AverageTime':'Duration'}, inplace=True)

In [ ]:
# drop the Unit column
df_clean.drop('Unit', axis=1, inplace=True)

In [ ]:
# Replace the image column
df_clean = df_clean[['Tour', 'Type', 'Duration', 'Price', 'Price Per', 'Cancellation','Tour By', 'City', 'Country', 'Image', 'Link to agency']]

In [ ]:
# limit a float to two decimal places
df_clean.Duration = round(df_clean.Duration, 2)

In [ ]:
# check the values
df_clean.Duration.value_counts()

In [ ]:
# check all the changes in the above section 2.5 Tour Duration column
df_clean.info()

In [ ]:
# scatter chart for the tour duration
x = df_clean['Duration'].value_counts().index
y = df_clean['Duration'].value_counts()
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20,10))
plt.scatter(x, y, color='#458B74')
plt.xticks( color ='#000000',fontsize=12)
plt.yticks(color ='#000000',fontsize=12)
plt.xlabel('Tour duration in hour',fontsize=20, color ='#000000')
plt.ylabel(' Number of tour',fontsize=20, color ='#000000')
plt.title('Tour Duration ',fontsize=22, color ='#000000')
plt.show();

*From the above plot, we can see the extrem value in the tour duration. With more than 100,000 hours*

In [ ]:
# finding the maximum value and drop it
df_clean['Duration'].max()

In [ ]:
# drop the extrem value
df_clean = df_clean[df_clean.Duration != 105408.0]

In [ ]:
# Statistic summary of price column
df_clean['Duration'].describe()

*Most of the Tours take 4-6 hours in average. Some of the tours are really long because it extends for days*

In [ ]:
# scatter chart for the tour duration after droping the extrem
x = df_clean['Duration'].value_counts().index
y = df_clean['Duration'].value_counts()
plt.style.use('seaborn-darkgrid')
plt.figure(figsize=(20,10))
plt.scatter(x, y, color='#458B74', s=100, alpha=0.7)
plt.xlim(0, 20)
plt.xticks( color ='#000000',fontsize=12)
plt.yticks(color ='#000000',fontsize=12)
plt.xlabel('Tour duration in hour',fontsize=20, color ='#000000')
plt.ylabel(' Number of tour',fontsize=20, color ='#000000')
plt.title('Tour Duration ',fontsize=22, color ='#000000')
plt.show();

*Most of the tours in this dataset are less than 5 hours long*

*2.6 Tour Price column*

In [ ]:
# removing the $ sign and , from ‘Price’ column using str.replace
df_clean['Price'] = df_clean['Price'].str.replace('$', '')
df_clean['Price'] = df_clean['Price'].str.replace(',', '')

In [ ]:
# convert the data type of Price to float
df_clean['Price'] = pd.to_numeric(df_clean['Price'], downcast="float")

In [ ]:
# limit a float to two decimal places
df_clean.Price = round(df_clean.Price, 2)

In [ ]:
# to check the above 
df_clean['Price'].dtype

In [ ]:
# Statistic summary of price column
df_clean['Price'].describe()

*The minimum tour price in this data is $3.35, whereas the maximum tour price is $12,570*

In [ ]:
# distribution plot for the price
plt.style.use('seaborn-darkgrid')
sns.set(font_scale=1.5)
plt.figure(figsize=(20,10))
sns.distplot(df_clean['Price'],color= ('#458B74'))
plt.title('The distribution of tour price',fontsize=22, color ='#000000');

*2.7 Price Per adult/group*

In [ ]:
# The average tour price based on per adult or per group
df_clean.groupby(['Price Per']).Price.agg(['mean'])

*Thte average price per group is much higher than per adult*

In [ ]:
# Scatter plot of the price per adult vs. price per group
plt.style.use('seaborn-darkgrid')
df_clean.plot(x='Price Per', y='Price', kind='scatter',figsize=(15,8), color='#458B74', s=100, alpha=0.5 , title='Price per Adult vs. Price per Group Based on Price');

*2.8 Cancellation column*

In [ ]:
# check the column values 
df_clean['Cancellation'].value_counts()

In [ ]:
# remove the LIKELY TO SELL OUT* value
df_clean = df_clean[df_clean.Cancellation != 'LIKELY TO SELL OUT*']

*All of the Tours in this dataset are flexible booking and provide free cancellation.*

*2.9 Tour By column*

In [ ]:
# number of unique value
df_clean['Tour By'].nunique()

*There are 3444 different agencies providing tours in this dataset*

*2.10 City & Country columns*

In [ ]:
# show number of unique values in each city & country column
print('Number of unoque value in City column:',df_clean['City'].nunique())
print('Number of unoque value in Country column:',df_clean['Country'].nunique())

In [ ]:
# reset the index
df_clean.reset_index(drop=True)

In [ ]:
# to check the last changes
df_clean.info()